In [39]:
import requests
import json
import psycopg2

# Database connection parameters
db_params = {
    "dbname": "weather_database",  
    "user": "vaibhav",             
    "password": "test123",   
    "host": "localhost",           
    "port": "5432"                 

def create_and_insert_table(data):
    try:
        # Establish a database connection
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # SQL statement to create the "weather_analysis" table
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS weather_analysis (
            id SERIAL PRIMARY KEY,
            place VARCHAR(255),
            country VARCHAR(255),
            weather VARCHAR(255),
            temp NUMERIC,
            max_temp NUMERIC,
            min_temp NUMERIC,
            wind_speed NUMERIC
        );
        """

        # Execute the SQL statement to create the table
        cursor.execute(create_table_sql)
        conn.commit()

        print("Table 'weather_analysis' created successfully.")

        # SQL statement to check for duplicate tables
        check_duplicate_sql = """
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables
            WHERE table_name = 'weather_analysis'
        );
        """

        cursor.execute(check_duplicate_sql)
        is_duplicate = cursor.fetchone()[0]

        if is_duplicate:
            print("Table 'weather_analysis' already exists.")
        else:
            print("Table 'weather_analysis' does not exist.")

        # Extract data from the 'data' dictionary
        place = data['name']
        country = data['sys']['country']
        weather = data['weather'][0]['main']
        temp = data['main']['temp']
        max_temp = data['main']['temp_max']
        min_temp = data['main']['temp_min']
        wind_speed = data['wind']['speed']

        # Insert extracted data into the table
        insert_sql = """
        INSERT INTO weather_analysis (place, country, weather, temp, max_temp, min_temp, wind_speed)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """

        # Execute the SQL statement to insert data
        cursor.execute(insert_sql, (place, country, weather, temp, max_temp, min_temp, wind_speed))
        conn.commit()

        print("Data inserted successfully.")

    except (Exception, psycopg2.Error) as error:
        print(f"Error: {error}")

    finally:
        # Close the cursor and database connection
        if conn:
            cursor.close()
            conn.close()

# Replace these variables with your API key and desired city/country
api_key = '6882adc00600840894cd93c205fbd356'
city = 'London'
country = 'uk'

# Get weather data
base_url = 'https://api.openweathermap.org/data/2.5/weather'
params = {
    'q': f'{city},{country}',
    'appid': api_key,
}
response = requests.get(base_url, params=params)
if response.status_code == 200:
    weather_data = response.json()
else:
    print("Error: Unable to fetch data from the API")

# Call the function to create, insert, and check for the table
create_and_insert_table(weather_data)


Table 'weather_analysis' created successfully.
Table 'weather_analysis' already exists.
Data inserted successfully.
